In [1]:
from pyomo.environ import Var, Expression, NonNegativeReals, Block, ConcreteModel, Constraint, Objective, Param, maximize, SolverFactory,RangeSet
import pyomo.environ as pyo
from idaes.core import FlowsheetBlock
import numpy as np
from pyomo.util.infeasible import log_infeasible_constraints, log_infeasible_bounds

In [80]:
# bw_vol = np.array([[0.05], [0.06]])
bw_vol = np.array([0.05, 0.06])
# bw_vol = bw_vol.T
dist =  np.array([[200,400,100],
                 [150,250,50]])

LCOW = np.array([0.2,0.1])

def transport_cost(bw_vol,dist):
    cost1 = (0.2 + 0.02*dist)*bw_vol
    cost2 = 0.02*dist
    return float(min(cost1,cost2))

demand = np.array([0.5,0.3,0.1])
# Assuming 3 demand sites
# Maximum supply possible
max_supply = sum(bw_vol)/3

print(bw_vol)
print(max_supply)


[0.05 0.06]
0.03666666666666667


In [81]:
def update_demand(supply_max,demand):
    for i in range(0,len(demand)):
        if demand[i]>supply_max:
            demand[i]=supply_max
    return demand

print(demand)
demand_new = update_demand(max_supply,demand)


print(demand_new)

[0.5 0.3 0.1]
[0.03666667 0.03666667 0.03666667]


In [82]:
# Trying to add powerplant as another index along with brackish water

m = ConcreteModel()

# transport_cost = 0.01
# demand = np.array([1,0.3,0.1])
# transport_coeff = bw_vol*dist*transport_cost(bw_vol,dist)

# brackish water site index
m.I = pyo.Set(initialize = range(dist.shape[0]))
# powerplant site index
m.J = pyo.Set(initialize = range(dist.shape[1]))

m.matrix= pyo.Set(initialize = m.I*m.J)
m.x = Var(m.matrix, domain=NonNegativeReals, bounds= (0,1))

m.constraints= pyo.ConstraintList()

# Sum of fractions should be less than or equal to 1
for i in m.I:
    m.constraints.add((sum(m.x[i,j] for j in m.J)) <= 1 )

# Production to be as close to maximum as possible
for j in m.J:
    m.constraints.add((sum(m.x[i,j]*float(bw_vol[i]) for i in m.I)) >= demand_new[j])

# for j in m.J:
#     m.constraints.add((sum(( LCOW[i]*m.x[i,j] + transport_cost(bw_vol[i],dist[i,j])*m.x[i,j] ) for i in m.I)) <= 1.4)

# def obj_demand(m):
#     return sum(sum(m.x[i,j]*float(bw_vol[i]) for i in m.I) for j in m.J)

# LCOW objective
def obj(m):
    return sum(sum((LCOW[i]*m.x[i,j] + transport_cost(bw_vol[i],dist[i,j])*m.x[i,j]) for i in m.I) for j in m.J)

m.obj1 = Objective(rule = obj)

solver = SolverFactory('ipopt')

results = solver.solve(m)

m.display()

Model unknown

  Variables:
    x : Size=6, Index=matrix
        Key    : Lower : Value                 : Upper : Fixed : Stale : Domain
        (0, 0) :     0 :   0.49999970193354454 :     1 : False : False : NonNegativeReals
        (0, 1) :     0 : 1.505903749979052e-08 :     1 : False : False : NonNegativeReals
        (0, 2) :     0 :    0.4999997028096596 :     1 : False : False : NonNegativeReals
        (1, 0) :     0 :   0.19444453148562377 :     1 : False : False : NonNegativeReals
        (1, 1) :     0 :    0.6111109362945155 :     1 : False : False : NonNegativeReals
        (1, 2) :     0 :   0.19444453210268886 :     1 : False : False : NonNegativeReals

  Objectives:
    obj1 : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 0.7019997633185657

  Constraints:
    constraints : Size=5
        Key : Lower               : Body                : Upper
          1 :                None :  0.9999994198022417 :   1.0
          2 :          

In [74]:
dict = (m.x.get_values())

# print(dict[(0,0)])
x = np.zeros(dist.shape)
# print(x)
for i,j in dict.keys():
    x[i,j] = dict[(i,j)]

x=np.array(x)
print(x)

[[4.99999702e-01 1.50590375e-08 4.99999703e-01]
 [1.94444531e-01 6.11110936e-01 1.94444532e-01]]


In [76]:
# check supply to each demand point

demand1 = bw_vol.T*x[:,0].T
demand2 = bw_vol.T*x[:,1].T
demand3 = bw_vol.T*x[:,2].T

print(x[:,0])
print(bw_vol)

print(sum(demand1))



[0.4999997  0.19444453]
[0.05 0.06]
0.03666665698581465
